In [16]:
import cv2
import base64
import ffmpeg
import ffmpeg._probe
import getpass
import glob
import json
import logging
import ntplib
import numpy as np
import os
import pandas as pd
import random
import re
import requests
import shutil
import subprocess
import sys
import time
import urllib.request
import winreg

from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from importlib import reload
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions

from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtNetwork import *
from PyQt5.QtWidgets import *
from PyQt5.QtMultimedia import *
from PyQt5.QtMultimediaWidgets import *

In [17]:
# Function to run ffmpeg command
def run_ffmpeg(cmd, bar_length=20):

    # Check if cmd is a list
    if not isinstance(cmd, list):
        raise TypeError('cmd must be a ffmpeg command list. Example: ["ffmpeg", "-i", "input.mp4", "-codec", "copy", "-y", "output.mp4"]')

    # Use subprocess to run the command and capture the output
    start_time = datetime.now().timestamp()
    p = subprocess.Popen(cmd, stderr=subprocess.PIPE)

    # Parse the output to get the progress
    while True:

        line = p.stderr.readline()
        if not line:
            break
        if b'Error' in line:  # Check for error lines
            print(line.decode())
            raise Exception('Error in ffmpeg command')
        if b'Duration' in line:
            duration = line.decode().split('Duration: ')[1].split(',')[0]
            hours, minutes, seconds = duration.split(':')
            total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
        if b'time=' in line:
            ftime = line.decode().split('time=')[1].split()[0]
            hours, minutes, seconds = ftime.split(':')
            current_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
            progress = current_seconds / total_seconds

            elapsed_time = datetime.now().timestamp() - start_time
            formatted_time = datetime.fromtimestamp(elapsed_time).strftime('%M:%S')
            progress_bar = '[' + '#' * int(progress * bar_length) + '-' * (20 - int(progress * 20)) + ']'
            print(f"Processing: {progress_bar} {progress:.2%} | {formatted_time}", end='\r')

    progress_bar = '[' + '#' * bar_length + ']'
    elapsed_time = datetime.now().timestamp() - start_time
    formatted_time = datetime.fromtimestamp(elapsed_time).strftime('%M:%S')
    print(f"Processing: {progress_bar} {1.:.2%} | {formatted_time}")
    
    # Wait for the command to finish and print the result
    p.wait()
    if p.returncode != 0:
        print('\nError:', p.stderr.read().decode())
    else:
        print('\nProcess completed successfully')

#### Class

In [18]:
from scipy.io import wavfile
from moviepy.video.io.VideoFileClip import VideoFileClip

class VidCleaner:

    def __init__(self, path, outdir=None):
        self.path = path
        if outdir is None:
            self.outdir = os.path.dirname(path)
        else:
            self.outdir = outdir

        self.outtmp = os.path.join(self.outdir, 'tmp')
        os.makedirs(self.outtmp, exist_ok=True)

        self.get_attributes()

        for file in os.listdir(self.outtmp):
            os.remove(f'{self.outtmp}/{file}')

    def get_attributes(self):

        self.basename = os.path.basename(self.path)
        self.filename = os.path.splitext(self.basename)[0]
        self.extension = os.path.splitext(self.basename)[1]
        self.size = os.path.getsize(self.path)
        
        video_capture = cv2.VideoCapture(self.path)
        self.fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        self.bitrate = int(video_capture.get(cv2.CAP_PROP_BITRATE))
        self.frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        self.duration = float(self.frame_count) / self.fps
        self.width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        video_capture.release()
        print(f'Name: {self.basename}')
        print(f'Width: {self.width}  Height: {self.height}  FPS: {self.fps}  Bitrate: {self.bitrate}  Duration: {self.duration:.2f}  Size: {self.size / 1024 / 1024:.2f} MB')
    
    def clean_video(self, interval=5):

        video_capture = cv2.VideoCapture(self.path)

        # Set the start and end frame positions for the first interval
        start_frame = 0
        end_frame = self.fps*interval

        # Loop through the time intervals
        while end_frame <= self.frame_count + self.fps*interval:
            # Extract the frames for the current interval
            frames = []
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            for i in range(start_frame, end_frame):
                success, frame = video_capture.read()
                if not success:
                    break
                frames.append(frame)
            frames = np.array(frames)

            # Save the frames as an MP4 file
            output_filepath = f'{self.outtmp}/{self.filename}-{end_frame//self.fps}.mp4'
            writer = cv2.VideoWriter(output_filepath, cv2.VideoWriter_fourcc(*'mp4v'), self.fps, (self.width, self.height))

            for frame in frames:
                writer.write(frame)
            writer.release()
            print(f'Saved: {output_filepath}', end='\r')

            # Update the start and end frame positions for the next interval
            start_frame = end_frame
            end_frame += self.fps*interval

        video_capture.release()

        # Concatenate the MP4 files
        files = os.listdir(self.outtmp)
        with open('filelist.txt', 'w') as f:
            for file in files:
                f.write(f"file '{self.outtmp}/{file}'\n")

        self.output_video = f'{self.outtmp}/{self.filename}-v.mp4'
        if os.path.exists(self.output_video): os.remove(self.output_video)

        cmd = ['ffmpeg', '-f', 'concat', '-safe', '0', '-i', 'filelist.txt', '-c', 'copy', self.output_video]
        subprocess.call(cmd)

        print(f'\nSaved: {self.output_video}')
        
        # Delete the temporary files
        os.remove('filelist.txt')
        for file in files:
            os.remove(f'{self.outtmp}/{file}')

    def clean_audio(self):
        
        clip = VideoFileClip(self.path)
        audio = clip.audio
        if audio == None:
            print(f'No audio found')
            return False

        sr = audio.fps
        print(f'Sample rate: {sr}')
        audio_list = np.array(audio.to_soundarray())
        clip.close()

        self.output_audio = f'{self.outtmp}/{self.filename}-a.wav'
        if os.path.exists(self.output_audio): os.remove(self.output_audio)
        wavfile.write(self.output_audio, sr, audio_list)
        print(f'Saved: {self.output_audio}')
        return True
        
    def clean(self, compress=True):

        self.output = f'{self.outdir}/{self.filename}-c.mp4'
        if os.path.exists(self.output):
            print(f'File already exists: {self.output}')
            return

        self.clean_video()
        hasAudio = self.clean_audio()

        self.output_raw = f'{self.outtmp}/{self.filename}-cr.mp4'
        if os.path.exists(self.output_raw): os.remove(self.output_raw)

        if hasAudio:
            cmd = ['ffmpeg', '-i', self.output_video, '-i', self.output_audio, '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', self.output_raw]
            run_ffmpeg(cmd)
            print(f'Saved: {self.output_raw}')
            os.remove(self.output_video)
            os.remove(self.output_audio)
        else:
            os.rename(self.output_video, self.output_raw)
            print(f'Copied: {self.output_raw}')

        if compress:
            cmd = ['ffmpeg', '-i', self.output_raw, '-c:a', 'copy', self.output]
            subprocess.call(cmd, shell=True)
            os.remove(self.output_raw)
        else:
            os.rename(self.output_raw, self.output)
        print(f'Saved: {self.output}')

        for file in os.listdir(self.outtmp):
            os.remove(f'{self.outtmp}/{file}')
        os.rmdir(self.outtmp)
        


In [19]:
class ImgCleaner:
    
    def __init__(self, path, outdir=None):
        self.path = path
        if outdir is None:
            self.outdir = os.path.dirname(path)
        else:
            self.outdir = outdir
        self.get_attributes()

    def get_attributes(self):

        self.basename = os.path.basename(self.path)
        self.filename = os.path.splitext(self.basename)[0]
        self.extension = os.path.splitext(self.basename)[1]
        self.size = os.path.getsize(self.path)
        
        img = cv2.imread(self.path)
        self.width = img.shape[1]
        self.height = img.shape[0]
        print(f'Name: {self.basename}')
        print(f'Width: {self.width}  Height: {self.height}  Size: {self.size / 1024 / 1024:.2f} MB')
    
    def clean(self, quality=75):

        self.output = f'{self.outdir}/{self.filename}-c.jpg'
        if os.path.exists(self.output):
            print(f'File already exists: {self.output}')
            return

        img = cv2.imread(self.path)
        flatimg = np.ravel(img)
        newimg = flatimg.reshape((self.height, self.width, 3))
        
        cv2.imwrite(self.output, newimg, [cv2.IMWRITE_JPEG_QUALITY, quality])
        print(f'Saved: {self.output}')
        

#### Mixer

In [20]:
import base64
from Crypto.Cipher import AES # pycryptodome

def mix(path, outdir=None, key=b'1234567890123456', iv=b'1234567890123456'):

    if len(key) != 16:
        raise ValueError('Key must be 16 bytes long')
    if len(iv) != 16:
        raise ValueError('IV must be 16 bytes long')

    with open(path, 'rb') as f:
        data = f.read()

    cipher = AES.new(key, AES.MODE_CBC, iv)
    padding_length = AES.block_size - len(data) % AES.block_size
    padded_data = data + (padding_length * chr(padding_length)).encode()
    encrypted_data = cipher.encrypt(padded_data)
    encoded_data = base64.b64encode(encrypted_data).decode()

    if outdir is None:
        outdir = os.path.dirname(path)

    output = f'{outdir}/{os.path.basename(path)}.txt'
    with open(output, 'w') as f:
        f.write(encoded_data)
    print(f'Mixed: {output}')


In [21]:
def unmix(path, outdir=None, key=b'1234567890123456', iv=b'1234567890123456'):

    if len(key) != 16:
        raise ValueError('Key must be 16 bytes long')
    if len(iv) != 16:
        raise ValueError('IV must be 16 bytes long')

    with open(path, 'r') as f:
        encoded_data = f.read()

    encrypted_data = base64.b64decode(encoded_data)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded_data = cipher.decrypt(encrypted_data)
    data = padded_data[:-padded_data[-1]]

    if outdir is None:
        outdir = os.path.dirname(path)

    output = f'{outdir}/{os.path.splitext(os.path.basename(path))[0]}'
    with open(output, 'wb') as f:
        f.write(data)
    print(f'Unmixed: {output}')


In [22]:
sys.exit()

SystemExit: 

#### Codec

In [ ]:
video = cv2.VideoCapture('')

# Get the codec information
fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
codec = chr(fourcc & 0xFF) + chr((fourcc >> 8) & 0xFF) + chr((fourcc >> 16) & 0xFF) + chr((fourcc >> 24) & 0xFF)

# Print the codec information
print("Codec: ", codec)

video.release()

#### Ops

In [23]:
in_dir = 'media'
out_dir = 'medio'

img_ext = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.tif', '.webp', '.heif', '.heic', '.bpg', '.svg', '.ico', '.eps', '.raw', '.cr2', '.nef', '.orf', '.sr2', '.raf', '.dng', '.arw', '.rw2', '.nrw', '.k25', '.kdc', '.dcr', '.mrw', '.pef', '.x3f', '.erf', '.srw', '.mef', '.mos', '.crw', '.dcs', '.3fr']
vid_ext = ['.mp4', '.avi', '.mov', '.wmv', '.flv', '.mkv', '.webm', '.m4v', '.mpg', '.mpeg', '.m2v', '.3gp', '.3g2', '.mxf', '.roq', '.ts', '.vob', '.gifv', '.rmvb', '.ogv', '.ogg', '.drc', '.mng', '.avi', '.mov', '.qt', '.wmv', '.yuv', '.rm', '.asf', '.amv', '.m4p', '.m4v', '.mpg', '.mp2', '.mpeg', '.mpe', '.mpv', '.mpg', '.mpeg', '.m2v', '.m4v', '.svi', '.3gp', '.3g2', '.mxf', '.roq', '.nsv', '.flv', '.f4v', '.f4p', '.f4a', '.f4b']
os.makedirs(out_dir, exist_ok=True)

for index, file in enumerate(os.listdir(in_dir)):
    print(f'Processing: {index + 1}/{len(os.listdir(in_dir))}')
    in_name = f'{in_dir}/{file}'
    extension = os.path.splitext(file)[1].lower()

    if extension in img_ext:
        out_name = f'{out_dir}/{os.path.splitext(file)[0]}-c.jpg'
        if os.path.exists(out_name):
            print(f'Already exists: {out_name}')
            continue
        clean = ImgCleaner(in_name, out_dir)
        clean.clean(80)
    elif extension in vid_ext:
        out_name = f'{out_dir}/{os.path.splitext(file)[0]}-c.mp4'
        if os.path.exists(out_name):
            print(f'Already exists: {out_name}')
            continue
        clean = VidCleaner(in_name, out_dir)
        clean.clean(80)
    else:
        raise ValueError(f'Unknown file type: {extension}')


Processing: 1/4
Name: stopwatch.mp4
Width: 1008  Height: 536  FPS: 30  Bitrate: 3264  Duration: 12.27  Size: 4.76 MB
Saved: medio\tmp/stopwatch-15.mp4
Saved: medio\tmp/stopwatch-v.mp4
Sample rate: 44100
Saved: medio\tmp/stopwatch-a.wav
Processing: [####################] 100.00% | 00:00

Process completed successfully
Saved: medio\tmp/stopwatch-cr.mp4
Saved: medio/stopwatch-c.mp4
Processing: 2/4
Name: Stage_x2.0.png
Width: 1452  Height: 966  Size: 0.41 MB
Saved: medio/Stage_x2.0-c.jpg
Processing: 3/4
Name: plane_resized.png
Width: 432  Height: 284  Size: 0.12 MB
Saved: medio/plane_resized-c.jpg
Processing: 4/4
Name: tom_resized.png
Width: 68  Height: 68  Size: 0.01 MB
Saved: medio/tom_resized-c.jpg


In [ ]:
in_dir = ''
out_dir = ''

os.makedirs(out_dir, exist_ok=True)
for index, file in enumerate(os.listdir(in_dir)):
    print(f'Processing: {index + 1}/{len(os.listdir(in_dir))}')
    in_name = f'{in_dir}/{file}'
    if os.path.exists(f'{in_name}.txt'):
        print(f'Already exists: {in_name}')
        continue
    mix(in_name, out_dir, key=b'0000000000000000', iv=b'0000000000000000')

In [ ]:
in_dir = ''
out_dir = ''

os.makedirs(out_dir, exist_ok=True)
for index, file in enumerate(os.listdir(in_dir)):
    print(f'Processing: {index + 1}/{len(os.listdir(in_dir))}')
    in_name = f'{in_dir}/{file}'
    out_name = f'{out_dir}/{os.path.splitext(file)[0]}'
    if os.path.exists(out_name):
        continue
    unmix(in_name, out_dir, key=b'0000000000000000', iv=b'0000000000000000')


#### GUI

In [24]:
class MainWindow(QWidget):

    global driver
    
    def __init__(self):
        super(MainWindow, self).__init__()
        self.initUI()

    def initUI(self):

        self.dir_select_box = QHBoxLayout()
        self.dir_select_button = QPushButton('Select Directory')
        self.dir_select_button.setFixedWidth(120)
        self.dir_select_button.clicked.connect(self.select_dir)
        self.dir_select = QLineEdit()
        self.dir_select.setFixedWidth(300)
        self.dir_select.setReadOnly(True)
        self.dir_select_box.addWidget(self.dir_select_button)
        self.dir_select_box.addWidget(self.dir_select)

        self.outdir_box = QHBoxLayout()
        self.outdir_button = QPushButton('Output Directory')
        self.outdir_button.setFixedWidth(120)
        self.outdir_button.clicked.connect(self.select_outdir)
        self.outdir = QLineEdit()
        self.outdir.setFixedWidth(300)
        self.outdir_box.addWidget(self.outdir_button)
        self.outdir_box.addWidget(self.outdir)

        self.key_box = QHBoxLayout()
        self.key_label = QLabel('Key')
        self.key = QLineEdit()
        self.key.setText('0000000000000000')
        self.key.textChanged.connect(self.key_changed)
        self.key_counter = QLabel()
        self.key_counter.setText('16/16')
        self.vertical = QLabel('|')
        self.iv_label = QLabel('IV')
        self.iv = QLineEdit()
        self.iv.setText('0000000000000000')
        self.iv.textChanged.connect(self.iv_changed)
        self.iv_counter = QLabel()
        self.iv_counter.setText('16/16')
        self.key_box.addWidget(self.key_label)
        self.key_box.addWidget(self.key)
        self.key_box.addWidget(self.key_counter)
        self.key_box.addWidget(self.vertical)
        self.key_box.addWidget(self.iv_label)
        self.key_box.addWidget(self.iv)
        self.key_box.addWidget(self.iv_counter)

        self.cmd_box = QHBoxLayout()
        self.mix_button = QPushButton('Mix')
        self.mix_button.clicked.connect(self.mix_clicked)
        self.unmix_button = QPushButton('Unmix')
        self.unmix_button.clicked.connect(self.unmix_clicked)
        self.vertical = QLabel('|')
        self.quality_button = QPushButton('Quality')
        self.quality_button.clicked.connect(self.clean_clicked)
        self.quality = QSpinBox()
        self.quality.setRange(0, 100)
        self.quality.setValue(80)
        self.cmd_run = QPushButton('Run')
        self.cmd_run.clicked.connect(self.run)
        self.cmd_box.addWidget(self.mix_button)
        self.cmd_box.addWidget(self.unmix_button)
        self.cmd_box.addWidget(self.vertical)
        self.cmd_box.addWidget(self.quality_button)
        self.cmd_box.addWidget(self.quality)
        self.cmd_box.addWidget(self.cmd_run)

        self.console = QTextEdit()
        self.console.setReadOnly(True)
        self.console.setFixedHeight(100)

        self.modeClean = True
        self.modeMix = False
        self.modeUnmix = False

        box = QVBoxLayout(self)
        box.addLayout(self.dir_select_box)
        box.addLayout(self.outdir_box)
        box.addLayout(self.cmd_box)
        box.addWidget(self.console)
        box.addLayout(self.key_box)

        self.setLayout(box)

        QApplication.setStyle(QStyleFactory.create('Fusion'))
        self.setWindowTitle("GUI")
        self.show()

    def select_dir(self):
        self.dir_select.setText(QFileDialog.getExistingDirectory(self, 'Select Directory'))
        if self.modeClean:
            if self.dir_select.text().endswith('-m') or self.dir_select.text().endswith('-u'):
                self.outdir.setText(self.dir_select.text()[:-2] + '-c')
            else:
                self.outdir.setText(self.dir_select.text() + '-c')
        elif self.modeMix:
            if self.dir_select.text().endswith('-c') or self.dir_select.text().endswith('-u'):
                self.outdir.setText(self.dir_select.text()[:-2] + '-m')
            else:
                self.outdir.setText(self.dir_select.text() + '-m')
        elif self.modeUnmix:
            if self.dir_select.text().endswith('-c') or self.dir_select.text().endswith('-m'):
                self.outdir.setText(self.dir_select.text()[:-2] + '-u')
            else:
                self.outdir.setText(self.dir_select.text() + '-u')

    def select_outdir(self):
        self.outdir.setText(QFileDialog.getExistingDirectory(self, 'Select Directory'))

    def clean_clicked(self):
        self.modeClean = True
        self.modeMix = False
        self.modeUnmix = False
        if self.dir_select.text().endswith('-m') or self.dir_select.text().endswith('-u'):
            self.outdir.setText(self.dir_select.text()[:-2] + '-c')
        else:
            self.outdir.setText(self.dir_select.text() + '-c')
        self.console.append('Clean mode selected')

    def mix_clicked(self):
        self.modeClean = False
        self.modeMix = True
        self.modeUnmix = False
        if self.dir_select.text().endswith('-c') or self.dir_select.text().endswith('-u'):
            self.outdir.setText(self.dir_select.text()[:-2] + '-m')
        else:
            self.outdir.setText(self.dir_select.text() + '-m')
        self.console.append('Mix mode selected')

    def unmix_clicked(self):
        self.modeClean = False
        self.modeMix = False
        self.modeUnmix = True
        if self.dir_select.text().endswith('-c') or self.dir_select.text().endswith('-m'):
            self.outdir.setText(self.dir_select.text()[:-2] + '-u')
        else:
            self.outdir.setText(self.dir_select.text() + '-u')
        self.console.append('Unmix mode selected')

    def mix(self):
        in_dir = self.dir_select.text()
        out_dir = self.outdir.text()

        os.makedirs(out_dir, exist_ok=True)
        for file in os.listdir(in_dir):
            in_name = f'{in_dir}/{file}'
            if os.path.exists(f'{in_name}.txt'):
                print(f'Already exists: {in_name}')
                continue
            key = self.key.text().encode()
            iv = self.iv.text().encode()
            mix(in_name, out_dir, key=key, iv=iv)

    def unmix(self):
        in_dir = self.dir_select.text()
        out_dir = self.outdir.text()

        os.makedirs(out_dir, exist_ok=True)
        for file in os.listdir(in_dir):
            in_name = f'{in_dir}/{file}'
            out_name = f'{out_dir}/{os.path.splitext(file)[0]}'
            if os.path.exists(out_name):
                continue
            key = self.key.text().encode()
            iv = self.iv.text().encode()
            unmix(in_name, out_dir, key=key, iv=iv)

    def clean(self):
        in_dir = self.dir_select.text()
        out_dir = self.outdir.text()

        img_ext = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.tif', '.webp', '.heif', '.heic', '.bpg', '.svg', '.ico', '.eps', '.raw', '.cr2', '.nef', '.orf', '.sr2', '.raf', '.dng', '.arw', '.rw2', '.nrw', '.k25', '.kdc', '.dcr', '.mrw', '.pef', '.x3f', '.erf', '.srw', '.mef', '.mos', '.crw', '.dcs', '.3fr']
        vid_ext = ['.mp4', '.avi', '.mov', '.wmv', '.flv', '.mkv', '.webm', '.m4v', '.mpg', '.mpeg', '.m2v', '.3gp', '.3g2', '.mxf', '.roq', '.ts', '.vob', '.gifv', '.rmvb', '.ogv', '.ogg', '.drc', '.mng', '.avi', '.mov', '.qt', '.wmv', '.yuv', '.rm', '.asf', '.amv', '.m4p', '.m4v', '.mpg', '.mp2', '.mpeg', '.mpe', '.mpv', '.mpg', '.mpeg', '.m2v', '.m4v', '.svi', '.3gp', '.3g2', '.mxf', '.roq', '.nsv', '.flv', '.f4v', '.f4p', '.f4a', '.f4b']
        
        os.makedirs(out_dir, exist_ok=True)
        for file in os.listdir(in_dir):

            in_name = f'{in_dir}/{file}'
            extension = os.path.splitext(file)[1].lower()
            
            if extension in img_ext:
                out_name = f'{out_dir}/{os.path.splitext(file)[0]}-c.jpg'
                if os.path.exists(out_name):
                    print(f'Already exists: {out_name}')
                    continue
                self.console.append(f'Cleaning: {in_name}')
                clean = ImgCleaner(in_name, out_dir)
                clean.clean(self.quality.value())
            elif extension in vid_ext:
                out_name = f'{out_dir}/{os.path.splitext(file)[0]}-c.mp4'
                if os.path.exists(out_name):
                    print(f'Already exists: {out_name}')
                    continue
                self.console.append(f'Cleaning: {in_name}')
                clean = VidCleaner(in_name, out_dir)
                clean.clean(self.quality.value())
            else:
                raise ValueError(f'Unknown file type: {extension}')
            
    def run(self):
        if not os.path.exists(self.dir_select.text()):
            self.console.append('Input directory does not exist')
            return
        
        if len(self.key.text()) != 16:
            self.console.append('Key must be 16 characters')
            return
        
        if len(self.iv.text()) != 16:
            self.console.append('IV must be 16 characters')
            return

        if self.modeClean:
            self.console.append(f'Cleaning: {self.dir_select.text()} to {self.outdir.text()}')
            time.sleep(0.5)
            self.clean()
        elif self.modeMix:
            self.console.append(f'Mixing: {self.dir_select.text()} to {self.outdir.text()}')
            time.sleep(0.5)
            self.mix()
        elif self.modeUnmix:
            self.console.append(f'Unmixing: {self.dir_select.text()} to {self.outdir.text()}')
            time.sleep(0.5)
            self.unmix()
        self.console.append('Done')

    def key_changed(self):
        self.key_counter.setText(f'{len(self.key.text())}/16')

    def iv_changed(self):
        self.iv_counter.setText(f'{len(self.iv.text())}/16')
            
app = QCoreApplication.instance()
if app is None:
    app = QApplication([])
window = MainWindow()
window.show()

try:
    from IPython.lib.guisupport import start_event_loop_qt5
    start_event_loop_qt5(app)
except ImportError:
    app.exec_()

Name: stopwatch.mp4
Width: 1008  Height: 536  FPS: 30  Bitrate: 3264  Duration: 12.27  Size: 4.76 MB
Saved: E:/Projects/FileTagger/media-c\tmp/stopwatch-15.mp4
Saved: E:/Projects/FileTagger/media-c\tmp/stopwatch-v.mp4
Sample rate: 44100
Saved: E:/Projects/FileTagger/media-c\tmp/stopwatch-a.wav
Processing: [####################] 100.00% | 00:00

Process completed successfully
Saved: E:/Projects/FileTagger/media-c\tmp/stopwatch-cr.mp4
Saved: E:/Projects/FileTagger/media-c/stopwatch-c.mp4
Name: Stage_x2.0.png
Width: 1452  Height: 966  Size: 0.41 MB
Saved: E:/Projects/FileTagger/media-c/Stage_x2.0-c.jpg
Name: plane_resized.png
Width: 432  Height: 284  Size: 0.12 MB
Saved: E:/Projects/FileTagger/media-c/plane_resized-c.jpg
Name: tom_resized.png
Width: 68  Height: 68  Size: 0.01 MB
Saved: E:/Projects/FileTagger/media-c/tom_resized-c.jpg


#### Rotate

In [ ]:
# Rotate 180 degrees
import subprocess
input_name = 'wen/wendy-cs.mp4'
output_name = 'wen/wendy-csp.mp4'
cmd = f'ffmpeg -i "{input_name}" -c:a copy "{output_name}"'
subprocess.call(cmd, shell=True)

In [ ]:
input_name = 'D:/Media/030618 Filtered/_Mega_/NBB/1com-c/4/kakaotalk_1543150421625-c.mp4'
output_name = 'D:/Media/030618 Filtered/_Mega_/NBB/1com-c/4/kakaotalk_1543150421625-c0.mp4'
# rotate video 180 with re encoding
cmd = ['ffmpeg', '-i', input_name, '-c:a', 'copy', '-vf', 'vflip,hflip', output_name]
subprocess.call(cmd)

0

In [25]:
class MainWindow(QWidget):

    global driver
    
    def __init__(self):
        super(MainWindow, self).__init__()
        self.initUI()

    def initUI(self):
        
        self.file_box = QHBoxLayout()
        self.file_select = QLineEdit()
        self.file_select.setReadOnly(True)
        self.suffix = QLineEdit('0')
        self.suffix.setFixedWidth(50)
        self.suffix.setAlignment(Qt.AlignRight)
        self.file_box.addWidget(self.file_select)
        self.file_box.addWidget(self.suffix)

        self.buttons_box = QHBoxLayout()
        self.select_button = QPushButton('Select File')
        self.select_button.clicked.connect(self.select_file)
        self.run_button = QPushButton('Rotate 180')
        self.run_button.clicked.connect(self.run)
        self.delete_checkbox = QCheckBox('Delete Original')
        self.delete_checkbox.setChecked(True)
        self.buttons_box.addWidget(self.select_button)
        self.buttons_box.addWidget(self.run_button)
        self.buttons_box.addWidget(self.delete_checkbox)

        box = QVBoxLayout(self)
        box.addLayout(self.file_box)
        box.addLayout(self.buttons_box)
        self.setLayout(box)

        QApplication.setStyle(QStyleFactory.create('Fusion'))
        self.setWindowTitle("Rotato")
        self.show()

    def select_file(self):
        file_name = QFileDialog.getOpenFileName(self, 'Select File')
        self.file_select.setText(file_name[0])

    def run(self):
        input_name = self.file_select.text()
        if input_name == '':
            return
        basename = os.path.basename(input_name)
        filename, ext = os.path.splitext(basename)
        output_name = f'{os.path.splitext(input_name)[0]}0.{ext}'
        cmd = f'ffmpeg -i "{input_name}" -c:a copy -vf "vflip,hflip" "{output_name}"'
        subprocess.call(cmd, shell=True)
        if self.delete_checkbox.isChecked():
            os.remove(input_name)
        self.file_select.setText('')


app = QCoreApplication.instance()
if app is None:
    app = QApplication([])
window = MainWindow()
window.show()

try:
    from IPython.lib.guisupport import start_event_loop_qt5
    start_event_loop_qt5(app)
except ImportError:
    app.exec_()